# Docker Swarm

## A critical walkthrough to make your life easier

    Jens Egholm Pedersen - jensegholm@protonmail.com

## Agenda

* Introduction
* The container paradigm
  * Hands on ``docker-machine``
* Orchestration
  * Hands on Docker swarm
* Use cases
  1. Buggy application with redundancy
  2. Monitoring and service discovery
* Recap

# Presentation formalia

* Who am I
* Who are you
* Presentation format and assumptions

## Who I am

    jensegholm@protonmail.com

### Education

* BSc. Political Science
* BSc. IT-University of Copenhagen
* MSc. IT & Cognition

### Professional experience

* Software engineer, CERN, Switzerland
* Assistant professor, Cphbusiness
* CTO Mobilized Construction

# Who you are

* Praqma

* Probably busy

![Moore's law](moore.png)

## Assumptions

* Our brains are bottlenecks
  * Computers are rapidly growing smarter
  * Humans are slowly growing smarter
  * Example: Assembly vs. NodeJS

* Technology should be human centric
  * Technology should adapt to us, not the other way around
* Tools should do one thing and do it well
* Text interfaces are superior 

## Context of this talk

* Copenhagen business academy (Cphbusiness)
* Course on large system development

* ~70 students
* 15 Hackernews clones
* 15 million messages $\approx$ 3 GB data
* Monitoring, logging, CD, scaling and security

* Clear goals driving relevant and intuitive teaching

# Presentation format

* Intuition
* Relevant
* Practical

* Theory
  * Listen, take notes, relate, ask
* Practice
  * Learn, internalise, criticise

* Metacognition
  * You learn better when you write down
  * Dunning-Kruger effect

![Donning-Kruger effect](donningkruger.png)

# Docker

* Docker machine
* Containers
* Software synchronisation
* Synchronisation in containers

## Docker machine

* A tool to manage virtual machines
  * Locally or remotely
  * Drivers for Virtualbox, AWS, Digital Ocean etc.

## Creating a machine

Syntax: ``docker-machine create --driver [virtualbox|digitalocean|...] [options] name``

Example: ``docker-machine create --driver virtualbox box1``

In [1]:
!docker-machine create --driver digitalocean --digitalocean-access-token=`cat dockertoken.txt` --digitalocean-region ams3 box1

Running pre-create checks...
Creating machine...
(box1) Creating SSH key...
(box1) Creating Digital Ocean droplet...
(box1) Waiting for IP address to be assigned to the Droplet...
Waiting for machine to be running, this may take a few minutes...
Detecting operating system of created instance...
Waiting for SSH to be available...
Detecting the provisioner...
Provisioning with ubuntu(systemd)...
Installing Docker...
Copying certs to the local machine directory...
Copying certs to the remote machine...
Setting Docker configuration on the remote daemon...
Checking connection to Docker...
Docker is up and running!
To see how to connect your Docker Client to the Docker Engine running on this virtual machine, run: docker-machine env box1


## Accessing a machine

    docker-machine ssh box1

In [26]:
! docker-machine ssh box1

ID                  NAME                MODE                REPLICAS            IMAGE                           PORTS
z9l0ap69rdy4        nodejs              replicated          1/1                 jegp/nodejs-prometheus:latest   *:8080->8080/tcp
ulcygre5mfn7        prometheus          replicated          1/1                 prom/prometheus:latest          *:9090->9090/tcp


## Accessing a machine from another computer

Copy environment from ``~/.docker``

    docker-machine env box1

In [4]:
!docker-machine env box1

export DOCKER_TLS_VERIFY="1"
export DOCKER_HOST="tcp://188.166.57.96:2376"
export DOCKER_CERT_PATH="/home/jens/.docker/machine/machines/box1"
export DOCKER_MACHINE_NAME="box1"
# Run this command to configure your shell: 
# eval $(docker-machine env box1)


## Deploying to a machine

    docker-machine ssh box1 docker run -p 80:80 -d nginx

In [ ]:
!docker-machine ssh box1 docker run -p 80:80 -d nginx

Unable to find image 'nginx:latest' locally
latest: Pulling from library/nginx
8176e34d5d92: Pulling fs layer
5b19c1bdd74b: Pulling fs layer
4e9f6296fa34: Pulling fs layer
4e9f6296fa34: Verifying Checksum
4e9f6296fa34: Download complete
8176e34d5d92: Verifying Checksum
8176e34d5d92: Download complete
5b19c1bdd74b: Verifying Checksum
5b19c1bdd74b: Download complete


## Listing machines

    docker-machine ls

In [6]:
!docker-machine ls

NAME   ACTIVE   DRIVER         STATE     URL                         SWARM   DOCKER        ERRORS
box1   -        digitalocean   Running   tcp://188.166.57.96:2376            v18.02.0-ce   
box2   -        digitalocean   Running   tcp://174.138.9.30:2376             v18.02.0-ce   
box3   -        digitalocean   Running   tcp://159.65.204.242:2376           v18.02.0-ce   


# Docker containers

* Simplification and isolition
  * Old, but good, idea

* Typical use case: microservices
  * Self-contained

* The container idea is _not_ perfect
  * Persistence (databases)
  * Synchronisation

* Containerisation is a paradigm
  * With paradigms comes incommensurability

## Occam's razor

* Docker is a framework for isolated containers
* Docker is incommensurable with high coupling/integration
* It forces us to deal with complexity elsewhere
  * Linux philosopy: do one thing and do it well

* This is just what we want from a container system!

* ... And what you will want from orchestration 

# Orchestration

* What do we want from orchestration?
* What do we _not_ want from orchestration?
* Docker swarm tutorial

## Orchestration requirements

* Efficient management
  * Deploy, teardown, updates (CRUD)

* Scheduling
  * Resource-aware

* Security
  * DMZ and hidden networks

## Orchestration unenviables

* Diffuse nomenclature
  * "Oups! I just X instead of Y" 

* Complexity
  * "Oups! Our cluster is down. Let me RTFM"
  * "I need to learn this. Please pay 50.000$ for a course"

* Time consuming
  * "Go to File -> Settings -> Advanced -> Update tab -> click on ..."
  * "I need to do X. Let me write a configuration file first"

## Docker swarm

* Efficient management: Purely CLI based
  * Docker as the atom
* Scheduling: Resource-aware and RAFT consensus
  * Semi-automatic
* Security: Public/private encryption
  * Encryption by default

## Docker swarm: concepts

* Nodes (in the cluster)
  * Worker
  * Manager

* Tasks
  * One single running container

* Services
  * A service provided by one or more containers

* Networks
  * Connecting one or more services

## Docker swarm nodes

* Managers
  * Maintains clusters and schedules services
  * Consensus: Have more than 2 for robust redundancy
* Workers


<div style="float: right">
![Docker swarm nodes](nodes.png)
</div>

## Docker swarm services

* Distributed application
  * Example: HTTPS server, database, etc.
* Based on a single docker container
  * Scaled across cluster
  
<div style="float: right">
![Docker swarm services](services.png)
</div>

## Starting a cluster

    docker swarm init

In [9]:
!docker-machine ssh box1 docker swarm init --advertise-addr 188.166.57.96

Swarm initialized: current node (1hp5ewkfro9zk39vzzimmhb1l) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join --token SWMTKN-1-1t2enc5n4m1hi6xijtlrrht5zcn0tlxnrdalbym918enr0gi1p-0w7fz61as8f5akfvz4knbm1xy 188.166.57.96:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.



## Joining a cluster

    docker swarm join --token ...

In [13]:
!docker-machine ssh box3 docker swarm join --token SWMTKN-1-1t2enc5n4m1hi6xijtlrrht5zcn0tlxnrdalbym918enr0gi1p-0w7fz61as8f5akfvz4knbm1xy 188.166.57.96:2377

This node joined a swarm as a worker.


## Listing nodes

    docker node ls

In [14]:
!docker-machine ssh box1 docker node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
1hp5ewkfro9zk39vzzimmhb1l *   box1                Ready               Active              Leader
nlpp2ydht5imcl9bs0we2r48i     box2                Ready               Active              
14ix82p9q6f5aml22rfzhbak9     box3                Ready               Active              
2w582xp0dq6jttzkg6puh7is5     jens-5              Down                Active              


* ``rm``, ``inspect`` etc.

* Note that the commands are following the linux philosophy

## Starting a service

    docker service create --name nginx nginx

In [15]:
!docker-machine ssh box1 docker service create --name nginx nginx

whllduur2tgtsc7nb7cynggzk
overall progress: 0 out of 1 tasks
1/1:  
overall progress: 0 out of 1 tasks
overall progress: 0 out of 1 tasks
overall progress: 0 out of 1 tasks
overall progress: 1 out of 1 tasks
verify: Waiting 5 seconds to verify that tasks are stable...
verify: Waiting 5 seconds to verify that tasks are stable...
verify: Waiting 5 seconds to verify that tasks are stable...
verify: Waiting 5 seconds to verify that tasks are stable...
verify: Waiting 5 seconds to verify that tasks are stable...
verify: Waiting 4 seconds to verify that tasks are stable...
verify: Waiting 4 seconds to verify that tasks are stable...
verify: Waiting 4 seconds to verify that tasks are stable...
verify: Waiting 4 seconds to verify that tasks are stable...
verify: Waiting 4 seconds to verify that tasks are stable...
verify: Waiting 3 seconds to verify that tasks are stable...
verify: Waiting 3 seconds to verify that tasks are stable...
verify: Waiting 3 seconds to verify that tasks are stable...

## Scaling a service

    docker service scale nginx=4

In [16]:
!docker-machine ssh box1 docker service scale nginx=4

nginx scaled to 4
overall progress: 0 out of 4 tasks
1/4:  
2/4:  
3/4:  
4/4:  
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall progress: 1 out of 4 tasks
overall p

## Listing services

    docker service ls

In [22]:
!docker-machine ssh box1 docker 


Usage:	docker node COMMAND

Manage Swarm nodes

Options:


Commands:
  demote      Demote one or more nodes from manager in the swarm
  inspect     Display detailed information on one or more nodes
  ls          List nodes in the swarm
  promote     Promote one or more nodes to manager in the swarm
  ps          List tasks running on one or more nodes, defaults to current node
  rm          Remove one or more nodes from the swarm
  update      Update a node

Run 'docker node COMMAND --help' for more information on a command.


## Recap

* Docker-machine
* Docker swarm 
  * Initialise clusters
  * Join clusters
  * Create services
  * Scaling services

* Networks 
* Configuration management
* Upgrading services

## Networks in Docker swarm

  * Overlay network
    * Networks as we know them (VPN)
  * Ingress network
    * Load-balancing

    docker network ls
    
    docker network create -d [overlay|ingress] name

In [25]:
    !docker-machine ssh box1 docker network create -d overlay mynginx

lesml5pz3nurhvyx60cuwne82


## Configuration in Docker swarm

* Configuration is shared between managers
* ``docker config``
  * ``docker config create name file``
  * Example: ``docker config create nginx.conf nginx.conf``
  

* Mounting configurations
  * Configurations can be mounted to files inside services
  * ``docker service create --name nginx nginx``
  * ``docker service create --config source=nginx.conf,target=/etc/nginx/nginx.conf --name prometheus prom/prometheus``

## Docker swarm ``update``

* Rolling upgrade
  * Image versions as upgrade tags
  * ``docker service update [service]``
  * Example: ``docker service update --image nginx:latest nginx``

* Also feasible with configuration, networks, etc.
  * ``docker service update --config-add source=nginx.conf,target=/etc/nginx/nginx.conf nginx``
  * ``docker service update --network-add mynetwork nginx``
  * ...

## Recap

* Instrumenting machines
  * ``docker-machine create``
* Starting Docker swarm clusters
  * ``docker swarm init``
  * ``docker swarm join --token ...``
* Creating Docker swarm services
  * ``docker service create --name nginx nginx``
* Scaling the service
  * ``docker service scale nginx=10``
* Updating a service
  * ``docker service update --image nginx:latest nginx``

## Docker swarm use cases

1. Buggy application
2. Monitoring with service discovery

## Use case 1/2: Buggy webserver

* Small Go webserver
  * Crashes somewhere between now and in 20 seconds


* https://github.com/Jegp/docker-swarm-go-example

1. ``docker service create --name webserver -p 8080:8080 helgecph/swarmserver``
  1. Launches a service with the very buggy go application
2. ``docker service scale webserver=5``
  1. Scales the service to 5 instances
3. Go to the webservice ``/status`` endpoint and behold the cluster self-heal
  1. ``curl http://$(docker-machine ip box2):8080/status``
  
... or use the ``create_cluster.sh`` shell script

## Use case 2/2: Monitoring with service discovery

* Nodejs service with unknown number of running tasks
* Prometheus for monitoring


* https://github.com/Jegp/docker-swarm-go-example


1. ``docker network create -d overlay monitoring``
  1. Create a common network for services that should be monitored
2. ``docker service create --name nodejs -p 8080:8080 --network monitoring jegp/nodejs-prometheus``
  1. Creates a NodeJS service from [Dockerhub](https://hub.docker.com/r/jegp/nodejs-prometheus/) on the ``monitoring`` network
3. ``docker config create prometheus prometheus.yml``
  1. Creates a configuration file for Prometheus searching for the 'nodejs' tasks
4. ``docker service create --name prometheus --network monitoring --config source=prometheus,target=/etc/prometheus/prometheus.yml -p 9090:9090 prometheus``
  1. Starts the prometheus service on the ``monitoring`` network using the configuration from step 3
5. ``docker service scale nodejs=10``
  1. Watch the prometheus cluster get more monitoring input as the service scales!

# Recap

* Status on Docker swarm
* Docker swarm versus Kubernetes
* Wrap-up

## Are we getting what we want?

* Efficient management? 
  * 4 commands = one service replicated 10 times
* Scheduling?
  * Free load balancing!
  * No automated scaling
* Security?
  * Networking
  * Automated public/private key pairings

## Comparing Docker swarm with Kubernetes

| Item | Docker | Kubernetes |
| ---------------------- | ------------------------ | -------------------------- |
| Interface | CLI mostly | CLI and configuration |
| Complexity | Simple | Hard |
| Time-to-market | Short | Medium | 
| Nomenclature | Easy | Hard |

## Docker stack

* Using docker compose files to configure clusters

* https://docs.docker.com/docker-cloud/apps/stacks/

## Wrap-up

* The technology bottleneck (you)
* Docker as a paradigm
* Docker swarm
* Use-cases
* Brief evaluation of Docker and Kubernetes



Thank you for your time!

### Contact

   Jens Egholm Pedersen - ``jensegholm@protonmail.com``